<a href="https://colab.research.google.com/github/yecatstevir/teambrainiac/blob/main/source/helper/hyperparameters_and_normalization_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grid Search Results
- Go to 'Runtime' in Colab browser bar, select 'Change Runtime Type', select 'High-RAM' from 'Runtime Shape'. 
- load local pickle file containing all GridSearchCV objects (single_subject_znorm_cv.pkl)
- Check results from GridSearchCV for each patient

### Mount Google Drive and clone repository
- open to source directory

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')#, force_remount = True)

Mounted at /content/gdrive


In [ ]:

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/yecatstevir/teambrainiac.git
# Change directory into cloned repo
%cd teambrainiac/source
!ls


Cloning into 'teambrainiac'...
remote: Enumerating objects: 588, done.
remote: Counting objects: 100% (588/588), done.
remote: Compressing objects: 100% (409/409), done.
remote: Total 588 (delta 356), reused 335 (delta 163), pack-reused 0
Receiving objects: 100% (588/588), 61.73 MiB | 41.86 MiB/s, done.
Resolving deltas: 100% (356/356), done.
/content/teambrainiac/source
AccuracyMeasures.ipynb	  __init__.py
cross_validation.py	  models
data			  process.py
DL			  SingleSubjectSVM.ipynb
Explore_data.ipynb	  SVM_Group_Child_Whole_Brain.ipynb
explore.py		  SVM_Group_YA_Whole_brain.ipynb
Group_All_MASK_SVM.ipynb  utils.py
helper			  VisualizationPlayground.ipynb
Images			  Visualize_Data.ipynb


### Load path_config.py 
- we are already in source so we can just load this file without chanding directory

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving path_config.py to path_config.py
User uploaded file "path_config.py" with length 196 bytes


### Load the individual clf's from the gridsearchcv
- filename single_subject_znorm_cv.pkl

### Import libraries


In [ ]:

# Import libraries
!pip install boto3 nilearn nibabel
from sklearn.model_selection import GridSearchCV
import pickle
from utils import data_to_nib, load_mat, open_pickle, access_load_data, save_data, create_mask, labels_mask_binary, masking_data, masked_data_n_labels
from sklearn.svm import SVC
import numpy as np
import pandas as pd
import random
import cv2 as cv
import altair as alt

### Load Data from AWS

In [ ]:
%%time
pkl_file = "single_subject_znorm_cv.pkl"
bool_mat = False
data = access_load_data(pkl_file, bool_mat)

CPU times: user 25 s, sys: 28.3 s, total: 53.3 s
Wall time: 1min 26s


In [ ]:
# Create dataframe from cross validation results
index = 0

cv_columns = ['user_id', 'score', 'C', 'kernel', 'random_state']
results = {}

for id in data.keys():
  user_clf = data[id]['model']
  ids = [id for x in range(len(user_clf.cv_results_['params']))]
  score = user_clf.cv_results_['mean_test_score']
  C = [x['C'] for x in user_clf.cv_results_['params']]
  kernel = [x['kernel'] for x in user_clf.cv_results_['params']]
  random_state = [x['random_state'] for x in user_clf.cv_results_['params']]
  for row in zip(ids, score, C, kernel, random_state):
    results[index] = row
    index += 1

df_results = pd.DataFrame.from_dict(results, orient='index', columns=cv_columns)
df_results.head() 

,user_id,score,C,kernel,random_state
0,10004_08693,0.702941,0.7,linear,111
1,10004_08693,0.702941,0.7,linear,222
2,10004_08693,0.832353,0.7,rbf,111
3,10004_08693,0.832353,0.7,rbf,222
4,10004_08693,0.702941,1.0,linear,111


## Looking at Cross Validation Results


In [ ]:
# Note that changing the random state did not have an effect on the score
# Deleting random state duplicates to simplify dataframe
df_results = df_results[df_results['random_state']==111]
df_results = df_results.drop(['random_state'], axis=1)
df_results.head()

,user_id,score,C,kernel
0,10004_08693,0.702941,0.7,linear
2,10004_08693,0.832353,0.7,rbf
4,10004_08693,0.702941,1.0,linear
6,10004_08693,0.869118,1.0,rbf
8,10004_08693,0.702941,5.0,linear


AttributeError: ignored

In [ ]:
df_results['c_kernel'] = ['C: ' + str(x[0]) + ',   kernel: ' + x[1] for x in zip(df_results['C'], df_results['kernel'])]

stripplot =  alt.Chart(df_results, width=40).mark_circle(size=25).encode(
    x=alt.X(
        'jitter:Q',
        title=None,
        axis=alt.Axis(values=[0], ticks=True, grid=False, labels=False),
        scale=alt.Scale(),
    ),
    y=alt.Y('score:Q'),
    color=alt.Color('c_kernel:N', legend=None),
    column=alt.Column(
        'c_kernel:N',
        header=alt.Header(
            labelAngle=90,
            titleOrient='bottom',
            labelOrient='bottom',
            labelAlign='right',
            labelPadding=3,
            labelFontSize=12
        ),
    ),
).transform_calculate(
    # Generate Gaussian jitter with a Box-Muller transform
    jitter='sqrt(-2*log(random()))*cos(2*PI*random())'
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
).properties(
    width = 70,
    height = 400
)

stripplot

alt.Chart(...)

In [ ]:
box_plot = alt.Chart(df_results).mark_boxplot(extent='min-max').encode(
    x=alt.X(
        'c_kernel:O',
        sort=alt.EncodingSortFieldFieldName(field='score', op='mean', order='descending')
    ),
    y='score:Q'
).properties(height=600, width=800)

box_plot

alt.Chart(...)

## Opening Second Pickle File

In [ ]:
# %%time
pkl_file = "single_subj_acc_measures.pkl"
bool_mat = False
data = access_load_data(pkl_file, bool_mat)

In [ ]:
data.keys()

dict_keys(['acc_df_psconly', 'acc_df_znorm_only', 'acc_df_znorm_psc', 'acc_df_nonorm'])

In [ ]:
index = 0
columns = ['normalization_type', 'subject_id', 'train_accuracy', 'validation_accuracy', 'test_accuracy']

dict_for_df = {}

for norm_type in data.keys():
  temp = data[norm_type]
  normalization_type = [norm_type for x in range(len(temp['Subject']))]
  subject_id = temp['Subject']
  train_accuracy = temp['TrainAcc']
  val_acc = temp['ValAcc']
  test_acc = temp['TestAcc']
  for x in np.array([normalization_type, subject_id, train_accuracy, val_acc, test_acc]).T:
    dict_for_df[index] = x
    index += 1

norm_df = pd.DataFrame.from_dict(dict_for_df, orient='index', columns=columns)
norm_df.head()

,normalization_type,subject_id,train_accuracy,validation_accuracy,test_accuracy
0,acc_df_psconly,10004_08693,0.904762,0.642857,0.785714
1,acc_df_psconly,10008_09924,0.761905,0.500000,0.500000
2,acc_df_psconly,10009_08848,0.964286,0.678571,0.511905
3,acc_df_psconly,10016_09694,0.988095,0.642857,0.488095
4,acc_df_psconly,10017_08894,0.761905,0.500000,0.595238


In [ ]:
train_box_plot = alt.Chart(norm_df).mark_boxplot(extent='min-max').encode(
    x=alt.X(
        'normalization_type:O',
        sort=alt.EncodingSortFieldFieldName(field='score', op='mean', order='descending')
    ),
    y='train_accuracy:Q'
)

val_box_plot = alt.Chart(norm_df).mark_boxplot(extent='min-max').encode(
    x=alt.X(
        'normalization_type:O',
        sort=alt.EncodingSortFieldFieldName(field='score', op='mean', order='descending')
    ),
    y='validation_accuracy:Q'
)

test_box_plot = alt.Chart(norm_df).mark_boxplot(extent='min-max').encode(
    x=alt.X(
        'normalization_type:O',
        sort=alt.EncodingSortFieldFieldName(field='score', op='mean', order='descending')
    ),
    y='test_accuracy:Q'
)

normalization_box_plot = train_box_plot | val_box_plot | test_box_plot
normalization_box_plot

alt.HConcatChart(...)